# Notebook para manejo de dataframes y generado de informacion a partir de la API de League of Legends.

https://developer.riotgames.com/docs/lol

Autores:  
* Rubén Fernández González (rfgonzalez13@esei.uvigo.es)
* Iván Montes Blanco (imblanco17@esei.uvigo.es)
* Jacobo Cortegoso Budiño (jcbudino17@esei.uvigo.es)
* Rubén Tejo Pereira (rtpereira18@esei.uvigo.es)






El objetivo es generar un Pandas dataframe o varios a partir de los J_SON de la api del LOL. Siendo los dataframes mas importantes lo de los campeones y los de las skills

El resultado final ideal para nuestro recomendador seria un dataframe con Campeones(nombre) como filas y las siguientes columnas: Rol, Lore, Q, W, E, R, Pasiva.

Importamos las librerias de pandas y json.
Necesitamos obtener el nombre de **todos** los campeones automáticamente, por eso usamos Data Dragon, la API de League of Legends, donde tenemos un objeto J_SON con todos los campeones que usaremos como índice para después.

In [1]:
import pandas as pd
import json

df = pd.read_json('http://ddragon.leagueoflegends.com/cdn/10.22.1/data/en_US/champion.json')
df.head()

,type,format,version,data
Aatrox,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Aatrox', 'key': ..."
Ahri,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Ahri', 'key': '1..."
Akali,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Akali', 'key': '..."
Alistar,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Alistar', 'key':..."
Amumu,champion,standAloneComplex,10.22.1,"{'version': '10.22.1', 'id': 'Amumu', 'key': '..."


Vemos que este DataFrame *df* contiene en su campo data el id de todos los campeones. Lo que hacemos ahora es iterar sobre el id para descargar de la API cada J_SON asociado a cada campeón y concatenarlo en un solo DataFrame.

El J_SON de cada campeón (como ejemplo mostramos a Yasuo) en Data Dragon tiene esta estructura:


In [2]:
EjemploCampeon = pd.read_json('http://ddragon.leagueoflegends.com/cdn/10.22.1/data/en_US/champion/Yasuo.json') 
EjemploCampeon = pd.read_json((EjemploCampeon['data']).to_json(), orient='index')
display(EjemploCampeon)

,id,key,name,title,image,skins,lore,blurb,allytips,enemytips,tags,partype,info,stats,spells,passive,recommended
Yasuo,Yasuo,157,Yasuo,the Unforgiven,"{'full': 'Yasuo.png', 'sprite': 'champion4.png...","[{'id': '157000', 'num': 0, 'name': 'default',...","An Ionian of deep resolve, Yasuo is an agile s...","An Ionian of deep resolve, Yasuo is an agile s...",[Dash through a minion to have Sweeping Blade ...,[Steel Tempest is very narrow. Dodge laterally...,"[Fighter, Assassin]",Flow,"{'attack': 8, 'defense': 4, 'magic': 4, 'diffi...","{'hp': 490, 'hpperlevel': 87, 'mp': 100, 'mppe...","[{'id': 'YasuoQ1Wrapper', 'name': 'Steel Tempe...","{'name': 'Way of the Wanderer', 'description':...","[{'champion': 'Yasuo', 'title': 'Beginner', 'm..."


Vemos que tenemos campos muy interesantes (y muchos otros despreciables) ya accesibles como el nombre y el lore. Sin embargo las descripciones de las skills o la pasiva estan dos niveles más abajo dentro del campo spell.

Bajamos esos niveles:

In [3]:
  s = EjemploCampeon['spells'][0]
  spellsData = [[s[0]['description'], s[1]['description'], s[2]['description'],s[3]['description'],EjemploCampeon['passive'][0]['description']]]
  EjemploHabilidades = pd.DataFrame(spellsData, columns = ['Q' , 'W' , 'E', 'R', 'Passive'], index = ['Yasuo'])
  display(EjemploHabilidades)

,Q,W,E,R,Passive
Yasuo,"Thrusts forward, damaging all enemies in a lin...",Creates a moving wall that blocks all enemy pr...,"Dashes through target enemy, dealing magic dam...",Blinks to an <factionIonia1>Airborne</factionI...,Yasuo's Critical Strike Chance is doubled. Add...


Concatenando el resultado de cada campeón y juntando los dataframes obtendremos el DataFrame *champInfo*. 

Sin embargo para el frontend de nuestra aplicacion necesitamos otros datos. Aprovechando la iteración sobre todos los campeones vamos a sacar un dataframe mas. Este dataframe contendrá el nombre de los campeones, el nombre de sus habilidades (Q,W,E,R, Passive) y la imagen de las mismas (Qimg, Wimg, Wimg, Eimg, Rimg, Passiveimg). Vamos a detallar el proceso con el Ejemplo de Yasuo:
<img src=http://ddragon.leagueoflegends.com/cdn/img/champion/splash/Yasuo_0.jpg>

In [4]:
spellsD = [[s[0]['name'], s[1]['name'], s[2]['name'],s[3]['name'], s[0]['id'], s[1]['id'], s[2]['id'], s[3]['id'], EjemploCampeon['passive'][0]['name'], EjemploCampeon['passive'][0]['image']['full']]]
EjemplofrontSpells = pd.DataFrame(spellsD, columns = ['Q' , 'W' , 'E', 'R', 'Qimg', 'Wimg', 'Eimg', 'Rimg', 'Passive', 'Passiveimg'], index = ['Yasuo'])
EjemplofrontSpells = pd.concat([EjemploCampeon['name'], EjemploCampeon['id'], EjemplofrontSpells], axis=1)
display(EjemplofrontSpells)

,name,id,Q,W,E,R,Qimg,Wimg,Eimg,Rimg,Passive,Passiveimg
Yasuo,Yasuo,Yasuo,Steel Tempest,Wind Wall,Sweeping Blade,Last Breath,YasuoQ1Wrapper,YasuoW,YasuoE,YasuoR,Way of the Wanderer,Yasuo_Passive.png


Ahora tenemos un DataFrame *EjemplofrontSpells* que contiene lo necesario para el frontend para mostrar a Yasuo en nuestra apicación web. Solo añadimos esto en un bucle para completarlo con todos los campeones. 

Las imágenes de los campeones funcionan con las siguientes rutas según su **id**:

(En ese caso el id = Yasuo)

*   Imagen Splash: http://ddragon.leagueoflegends.com/cdn/img/champion/splash/Yasuo_0.jpg
*   Imagen Loading: http://ddragon.leagueoflegends.com/cdn/img/champion/loading/Yasuo_0.jpg
*   Imagen miniatura: http://ddragon.leagueoflegends.com/cdn/10.22.1/img/champion/Yasuo.png

Las imagenes de las habilidades usan **Ximg** y la passiva **Passiveimg**

*   http://ddragon.leagueoflegends.com/cdn/10.22.1/img/spell/YasuoQ1Wrapper.png
*   http://ddragon.leagueoflegends.com/cdn/10.22.1/img/passive/Yasuo_Passive.png





Ahora solo queda generar los dos DataFrames completos con todos los campeones. Recordemos que teníamos el DataFrame *df* que básicamente contiene el 'id' de todos ellos. Iterando sobre el mismo podemos acceder a cada DataDragon de campeón y extraer la información que deseamos. El resultado debería ser *champInfo* para nuestro recomendador y *frontInfo* para el front end de nuestra aplicación.

In [5]:
champInfo = pd.DataFrame() 
frontInfo = pd.DataFrame()

for i in df['data']:
  #pasar json campeones a dataframe
  leeJson = pd.read_json('http://ddragon.leagueoflegends.com/cdn/10.22.1/data/en_US/champion/'+i['id']+'.json')
  datosChamp = pd.read_json((leeJson['data']).to_json(), orient='index')
  
  #pasar json spells y passive a dataframe
  s = datosChamp['spells'][0]
  
  #champInfo
  spellsData = [[s[0]['description'], s[1]['description'], s[2]['description'],s[3]['description'],datosChamp['passive'][0]['description']]]
  spells = pd.DataFrame(spellsData, columns = ['Q', 'W' , 'E', 'R', 'Passive'], index = [i['id']])

  #juntar dataframes y añadir a resultado
  toappend = pd.concat([datosChamp, spells], axis=1)
  champInfo = champInfo.append(toappend)

  #frontInfo
  spellsD = [[s[0]['name'], s[1]['name'], s[2]['name'],s[3]['name'], s[0]['id'], s[1]['id'], s[2]['id'], s[3]['id'], datosChamp['passive'][0]['name'], datosChamp['passive'][0]['image']['full']]]
  front = pd.DataFrame(spellsD, columns = ['Q' , 'W' , 'E', 'R', 'Qimg', 'Wimg', 'Eimg', 'Rimg', 'Passive', 'Passiveimg'], index = [i['id']])
  
  front = pd.concat([datosChamp['name'], datosChamp['id'], front], axis=1)
  frontInfo = frontInfo.append(front)
#end of loop

In [ ]:
frontInfo.head()

,name,id,Q,W,E,R,Qimg,Wimg,Eimg,Rimg,Passive,Passiveimg
Aatrox,Aatrox,Aatrox,The Darkin Blade,Infernal Chains,Umbral Dash,World Ender,AatroxQ,AatroxW,AatroxE,AatroxR,Deathbringer Stance,Aatrox_Passive.png
Ahri,Ahri,Ahri,Orb of Deception,Fox-Fire,Charm,Spirit Rush,AhriOrbofDeception,AhriFoxFire,AhriSeduce,AhriTumble,Essence Theft,Ahri_SoulEater2.png
Akali,Akali,Akali,Five Point Strike,Twilight Shroud,Shuriken Flip,Perfect Execution,AkaliQ,AkaliW,AkaliE,AkaliR,Assassin's Mark,Akali_P.png
Alistar,Alistar,Alistar,Pulverize,Headbutt,Trample,Unbreakable Will,Pulverize,Headbutt,AlistarE,FerociousHowl,Triumphant Roar,Alistar_E.png
Amumu,Amumu,Amumu,Bandage Toss,Despair,Tantrum,Curse of the Sad Mummy,BandageToss,AuraofDespair,Tantrum,CurseoftheSadMummy,Cursed Touch,Amumu_Passive.png


In [ ]:
champInfo.head()

,id,key,name,title,image,skins,lore,blurb,allytips,enemytips,tags,partype,info,stats,spells,passive,recommended,Q,W,E,R,Passive
Aatrox,Aatrox,266,Aatrox,the Darkin Blade,"{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[{'id': '266000', 'num': 0, 'name': 'default',...",Once honored defenders of Shurima against the ...,Once honored defenders of Shurima against the ...,[Use Umbral Dash while casting The Darkin Blad...,"[Aatrox's attacks are very telegraphed, so use...","[Fighter, Tank]",Blood Well,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl...","[{'id': 'AatroxQ', 'name': 'The Darkin Blade',...","{'name': 'Deathbringer Stance', 'description':...","[{'champion': 'Aatrox', 'title': 'AatroxARAM',...","Aatrox slams his greatsword down, dealing phys...","Aatrox smashes the ground, dealing damage to t...","Passively, Aatrox heals when damaging enemy ch...","Aatrox unleashes his demonic form, fearing nea...","Periodically, Aatrox's next basic attack deals..."
Ahri,Ahri,103,Ahri,the Nine-Tailed Fox,"{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[{'id': '103000', 'num': 0, 'name': 'default',...",Innately connected to the latent power of Rune...,Innately connected to the latent power of Rune...,"[Use Charm to set up your combos, it will make...",[Ahri's survivability is dramatically reduced ...,"[Mage, Assassin]",Mana,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mppe...","[{'id': 'AhriOrbofDeception', 'name': 'Orb of ...","{'name': 'Essence Theft', 'description': 'When...","[{'champion': 'Ahri', 'title': 'AhriARAM', 'ma...","Ahri sends out and pulls back her orb, dealing...",Ahri gains a brief burst of movement speed and...,Ahri blows a kiss that damages and charms an e...,"Ahri dashes forward and fires essence bolts, d...",When Ahri strikes 9 enemies with her abilities...
Akali,Akali,84,Akali,the Rogue Assassin,"{'full': 'Akali.png', 'sprite': 'champion0.png...","[{'id': '84000', 'num': 0, 'name': 'default', ...",Abandoning the Kinkou Order and her title of t...,Abandoning the Kinkou Order and her title of t...,[Akali excels at killing fragile champions. Le...,[Akali can still be hit by area effect spells ...,[Assassin],Energy,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'hp': 575, 'hpperlevel': 95, 'mp': 200, 'mppe...","[{'id': 'AkaliQ', 'name': 'Five Point Strike',...","{'name': 'Assassin's Mark', 'description': 'De...","[{'champion': 'Akali', 'title': 'AkaliARAM', '...","Akali throws out five kunai, dealing damage ba...",Akali drops a cover of smoke and briefly gains...,"Flip backward and fire a shuriken forward, dea...","Akali leaps in a direction, damaging enemies s...",Dealing spell damage to a champion creates a r...
Alistar,Alistar,12,Alistar,the Minotaur,"{'full': 'Alistar.png', 'sprite': 'champion0.p...","[{'id': '12000', 'num': 0, 'name': 'default', ...",Always a mighty warrior with a fearsome reputa...,Always a mighty warrior with a fearsome reputa...,[Using Pulverize can allow you to establish be...,[Alistar is very disruptive but very tough - t...,"[Tank, Support]",Mana,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","{'hp': 600, 'hpperlevel': 106, 'mp': 350, 'mpp...","[{'id': 'Pulverize', 'name': 'Pulverize', 'des...","{'name': 'Triumphant Roar', 'description': 'Al...","[{'champion': 'Alistar', 'title': 'AlistarARAM...","Alistar smashes the ground, dealing damage to ...","Alistar rams a target with his head, dealing d...","Alistar tramples nearby enemy units, ignoring ...","Alistar lets out a wild roar, removing all cro...",Alistar charges his roar by stunning or displa...
Amumu,Amumu,32,Amumu,the Sad Mummy,"{'full': 'Amumu.png', 'sprite': 'champion0.png...","[{'id': '32000', 'num': 0, 'name': 'default', ...",Legend claims that Amumu is a lonely and melan...,Legend claims that Amumu is a lonely and melan...,"[Amumu is highly dependent on teammates, so tr...",[Avoid bunching up with other allies w

Tenemos los dos DataFrames listos aunque *champInfo* contiene demasiados campos que para nuestro reconmendador son totalmente innecesarios, asi que los eliminamos:

In [6]:
del champInfo['skins']
del champInfo['key']
del champInfo['id']
del champInfo['blurb']
del champInfo['title']
del champInfo['allytips']
del champInfo['enemytips']
del champInfo['info']
del champInfo['recommended']
del champInfo['spells']
del champInfo['passive']
del champInfo['stats']
del champInfo['image']

champInfo.head()

,name,lore,tags,partype,Q,W,E,R,Passive
Aatrox,Aatrox,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,"Aatrox slams his greatsword down, dealing phys...","Aatrox smashes the ground, dealing damage to t...","Passively, Aatrox heals when damaging enemy ch...","Aatrox unleashes his demonic form, fearing nea...","Periodically, Aatrox's next basic attack deals..."
Ahri,Ahri,Innately connected to the latent power of Rune...,"[Mage, Assassin]",Mana,"Ahri sends out and pulls back her orb, dealing...",Ahri gains a brief burst of movement speed and...,Ahri blows a kiss that damages and charms an e...,"Ahri dashes forward and fires essence bolts, d...",When Ahri strikes 9 enemies with her abilities...
Akali,Akali,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy,"Akali throws out five kunai, dealing damage ba...",Akali drops a cover of smoke and briefly gains...,"Flip backward and fire a shuriken forward, dea...","Akali leaps in a direction, damaging enemies s...",Dealing spell damage to a champion creates a r...
Alistar,Alistar,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana,"Alistar smashes the ground, dealing damage to ...","Alistar rams a target with his head, dealing d...","Alistar tramples nearby enemy units, ignoring ...","Alistar lets out a wild roar, removing all cro...",Alistar charges his roar by stunning or displa...
Amumu,Amumu,Legend claims that Amumu is a lonely and melan...,"[Tank, Mage]",Mana,"Amumu tosses a sticky bandage at a target, stu...","Overcome by anguish, nearby enemies lose a per...",Permanently reduces the physical damage Amumu ...,Amumu entangles surrounding enemy units in ban...,Amumu's basic attacks <font color='#9b0f5f'>Cu...


**NOTA**: Para seleccionar un dato del cualquier DataFrame usaremos DataFrame.at[fila,columna].

In [7]:
print(frontInfo.at[ 'Yasuo' , 'Q' ])

Steel Tempest


# Recomendador
Necesitamos un Dataframe con nombre, todas las skills concatenadas y el lore (que ya teníamos).

In [8]:
6#sacar textos preprocesados de las habilidades y lore para sistema de recomendacion


#importar librerias
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

#procesar texto habilidades para cada campeon
ps = PorterStemmer()
bags = pd.DataFrame(columns=['name','skill','lore'])

for i in frontInfo["id"]:
    stops = set(stopwords.words("english"))
    #extraer y procesar informacion sobre skills
    habil = word_tokenize(champInfo.at[i,'Q']+champInfo.at[i,'W']+champInfo.at[i,'E']+champInfo.at[i,'R'])
    habil = [ps.stem(w) for w in habil if not w in stops and w.isalnum()]
    habil = " ".join(habil)
    #extraer y procesar informacion sobre lore
    lore = word_tokenize(champInfo.at[i,'lore'])
    lore = [ps.stem(w) for w in lore if not w in stops and w.isalnum()]
    lore = " ".join(lore)
    #juntar y añadir a dataframe
    bags = bags.append({'name':champInfo.at[i,'name'],'skill':habil,'lore':lore},ignore_index=True)

display(bags)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,name,skill,lore
0,Aatrox,aatrox slam greatsword deal physic damag He sw...,onc honor defend shurima void aatrox brethren ...
1,Ahri,ahri send pull back orb deal magic damag way t...,innat connect latent power runeterra ahri vast...
2,Akali,akali throw five kunai deal damag base bonu at...,abandon kinkou order titl fist shadow akali st...
3,Alistar,alistar smash ground deal damag nearbi enemi t...,alway mighti warrior fearsom reput alistar see...
4,Amumu,amumu toss sticki bandag target stun damag tar...,legend claim amumu lone melancholi soul ancien...
...,...,...,...
147,Zed,zed shadow throw br br each shuriken deal dama...,utterli ruthless without merci zed leader orde...
148,Ziggs,zigg throw bounc bomb deal magic fling explos ...,with love big bomb short fuse yordl zigg explo...
149,Zilean,toss bomb target area stick unit come near pri...,onc power icathian mage zilean becam obsess pa...
150,Zoe,zoe fire missil redirect flight deal damag lon...,As embodi mischief imagin chang zoe act cosmic...


Aplicar *bag of words* a los dos campos y generear los vectores para el recomendados

In [10]:
#pasar dataframe texto preprocesado a bag of words y añadir a dataframe
from sklearn.feature_extraction.text import TfidfVectorizer
bagOfWordsModel = TfidfVectorizer()
vect_bag_skills = []
vect_bag_lore = []

#bag of words de skills y lore
bagOfWordsModel.fit(bags['skill'])
bag_skills = bagOfWordsModel.transform(bags['skill'])

#bag of words de lore
bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(bags['lore'])
bag_lore = bagOfWordsModel.transform(bags['lore'])

#generar vectores con los bags de skill y lore
for i in bag_skills:
  vect_bag_skills.append(i)
for i in bag_lore:
  vect_bag_lore.append(i)

#añadir vectores como columna al dataframe
bags['bag_skill'] = vect_bag_skills
bags['bag_lore'] = vect_bag_lore
display(bags)

,name,skill,lore,bag_skill,bag_lore
0,Aatrox,aatrox slam greatsword deal physic damag He sw...,onc honor defend shurima void aatrox brethren ...,"(0, 1357)\t0.11745865911977424\n (0, 1307)\...","(0, 2466)\t0.11957558735673952\n (0, 2439)\..."
1,Ahri,ahri send pull back orb deal magic damag way t...,innat connect latent power runeterra ahri vast...,"(0, 1407)\t0.3091455263021661\n (0, 1399)\t...","(0, 2349)\t0.17465849933115274\n (0, 2214)\..."
2,Akali,akali throw five kunai deal damag base bonu at...,abandon kinkou order titl fist shadow akali st...,"(0, 1415)\t0.11049841026935704\n (0, 1368)\...","(0, 2416)\t0.12447302451809314\n (0, 2194)\..."
3,Alistar,alistar smash ground deal damag nearbi enemi t...,alway mighti warrior fearsom reput alistar see...,"(0, 1422)\t0.18820759156730632\n (0, 1356)\...","(0, 2416)\t0.13582511527225036\n (0, 2406)\..."
4,Amumu,amumu toss sticki bandag target stun damag tar...,legend claim amumu lone melancholi soul ancien...,"(0, 1435)\t0.13829057594843785\n (0, 1357)\...","(0, 2468)\t0.1677688047071753\n (0, 2462)\t..."
...,...,...,...,...,...
147,Zed,zed shadow throw br br each shuriken deal dama...,utterli ruthless without merci zed leader orde...,"(0, 1452)\t0.5923182988051594\n (0, 1413)\t...","(0, 2498)\t0.3577721694045413\n (0, 2456)\t..."
148,Ziggs,zigg throw bounc bomb deal magic fling explos ...,with love big bomb short fuse yordl zigg explo...,"(0, 1456)\t0.1017313074683368\n (0, 1454)\t...","(0, 2502)\t0.34248585930095526\n (0, 2495)\..."
149,Zilean,toss bomb target area stick unit come near pri...,onc power icathian mage zilean becam obsess pa...,"(0, 1356)\t0.1523508693422858\n (0, 1316)\t...","(0, 2503)\t0.3160530927279859\n (0, 2452)\t..."
150,Zoe,zoe fire missil redirect flight deal damag lon...,As embodi mischief imagin chang zoe act cosmic...,"(0, 1455)\t0.20019837449093777\n (0, 1413)\...","(0, 2504)\t0.4177284028921276\n (0, 2462)\t..."


Recomendador, usamos distancias coseno para las skills y euclídeas para el lore. A base de prueba y error son con las que mejores resultados obtuvimos.

In [11]:
#obtener campeones similares dado un nombre
from sklearn.metrics import pairwise_distances

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def recomendar(nombre):
  #obtener posicion del campeon buscado
  pos = -1;
  for i in bags.index:
    if (bags.at[i,'name']==nombre):
      pos = i;
      break;
  
  #generar vectores con distancias
  vect_skill = []
  vect_lore = []
  for i in bags.index:
    #calcular distancias de vectores
    vect_skill.append(pairwise_distances(bags.at[i,'bag_skill'],bags.at[pos,'bag_skill'],metric='cosine')[0][0])
    vect_lore.append(pairwise_distances(bags.at[i,'bag_lore'],bags.at[pos,'bag_lore'],metric='euclidean')[0][0])

  #vector similaridad habilidades
  distance_scores = list(enumerate(vect_skill))
  ordered_scores = sorted(distance_scores, key=lambda x: x[1])
  skills_sim = ordered_scores[1:11]

  #vector similaridad lore
  distance_scores = list(enumerate(vect_lore))
  ordered_scores = sorted(distance_scores, key=lambda x: x[1])
  lore_sim = ordered_scores[0:10]

  #mostrar campeones similares segun skill
  print("> Segun skills: ")
  count = 0;
  lista_skills = []
  for i in skills_sim:
    count += 1;
    name = bags.at[i[0],'name'];
    print(str(count) + " : " + name)
    lista_skills.append(name)
    #display(champInfo.iloc[ i[0] , : ])
  
  #mostrar campeones similares segun lore
  print("> Segun lore: ")
  count = 0;
  lista_lore = []
  for i in lore_sim:
    count += 1;
    name = bags.at[i[0],'name'];
    print(str(count) + " : " + name)
    lista_lore.append(name)
    #display(champInfo.iloc[ i[0] , : ])
  print("Interseccion:")
  print(intersection(lista_lore, lista_skills))

  


recomendar('Sona')

> Segun skills: 
1 : Nami
2 : Lulu
3 : Taric
4 : Seraphine
5 : Rakan
6 : Rumble
7 : Kassadin
8 : Camille
9 : Soraka
10 : Sivir
> Segun lore: 
1 : Sona
2 : Galio
3 : Fizz
4 : Akali
5 : Rakan
6 : Vayne
7 : Poppy
8 : Fiora
9 : Tahm Kench
10 : Karma
Interseccion:
['Rakan']


# Valorador

Recoger tweets recientes de un campeon, por ejemplo Qiyana.

In [ ]:
import os
import tweepy as tw
import pandas as pd

num = 0
Losdatos = pd.read_csv('TweetsValorados.csv')
del Losdatos[Losdatos.columns[0]]

consumer_key= 'KatBRlOrrS5Ocj1spbU4kY3Nf'
consumer_secret= 'GLcJQOSrNi2btYSu8UUqaVVpKcxLnMdjB9ffkU0LtauxPpjOoL'
access_token= '1337738322945462272-nR5TSl5hGrTojizeznHRLOOAuKb20t'
access_token_secret= 'F4qVZ7icDX6Gpiqa7Z1DSIXUuAdvbTXCXqMVfooEL2WTP'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

search_words = "Rammus" + " -filter:retweets"
date_since = "2020-10-12"

tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(250)


tuits = [tweet.text for tweet in tweets]
display(tuits)
tweet_text = pd.DataFrame(data=tuits,columns=['tweet'])
tweet_text

['@im_hawkward Rammus!!! Brilliant ty',
 '@TheGOATRammus @TFT 1 cost rammus with built in zrot would be dope.',
 'Yummi + rammus do jr e do racca = gabigol e BH 2019',
 '@LoveLilliana Malphite, Mundo, rammus, maokai, yuumi, master yi, Annie would be the only midlander and idfk for arc',
 '@hotboy_goldie no way these niggas turned into rammus. https://t.co/cKpapvYnfS',
 '@ZitZitovna @mehlik_ rammus https://t.co/iW1yb2kDZY',
 "@MarkYetter And also, remember that you all intend for this game to last another decade. So you'll to do it at some… https://t.co/hsyiUeWiew",
 'Imagina bard mid, yummi jungle e rammus top e dar stomp!!! btw o jungler estava com o rato na mao esquerda!!!… https://t.co/OpWTDFUfWN',
 '@zefyuuto Rammus',
 '@JesterX19 @TFT Both teemo and voli has been in the game. Only rammus has not.',
 '@Enchaladastyle @CorinnaKopf Hey miah, just wanted to say thanks for carrying me all the way from iron iii to diamo… https://t.co/rKQQkEOOeW',
 '@BoyProfanity @AerialQueenLiz Playing 

,tweet
0,@im_hawkward Rammus!!! Brilliant ty
1,@TheGOATRammus @TFT 1 cost rammus with built i...
2,Yummi + rammus do jr e do racca = gabigol e BH...
3,"@LoveLilliana Malphite, Mundo, rammus, maokai,..."
4,@hotboy_goldie no way these niggas turned into...
...,...
203,@googasofficial @aposefive if i play rammus i ...
204,@aposefive @HayLightz just play rammus
205,@El_Cardoso_ Rammus E
206,@1v9Jonas NAH BUT fuck rammus players


In [ ]:
import ipywidgets as widgets

eltweet = tuits[num]
num = num +1
if num > len(tuits):
  num = 0
  print("CUIDAAAOOO")
print(eltweet)
resultado = widgets.ToggleButtons(
    options=['Positivo', 'Negativo', 'Descartar'],
    description='Valoracion:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

resultado

@buycapes @kumarighalex Insta locking amumu or rammus just so someone else feels my pain


ToggleButtons(description='Valoracion:', options=('Positivo', 'Negativo', 'Descartar'), tooltips=('Description…

In [ ]:
if resultado.value.lower() == "positivo":
  Losdatos.loc[len(Losdatos)]=[eltweet,1]
  Losdatos.to_csv('TweetsValorados.csv')
  print("Guardado positivo")
if resultado.value.lower() == "negativo":
  Losdatos.loc[len(Losdatos)]=[eltweet,0]  
  Losdatos.to_csv('TweetsValorados.csv')
  print("Guardado negativo")
display(Losdatos)

Guardado positivo


,tweet,valoracion
0,THIS MASTER YI DAMAGE IS JUST REALLY CRAZY! – ...,1
1,@HangCJay there’s just something about master ...,0
2,A master yi complaining about how lee sin is a...,1
3,Master Yi takes skill https://t.co/2U9bH2U5Sc,1
4,@samuelgh1998 @G2esports Y bannear a master yi,1
5,@eademo1 perma ban for all kassadin picks* im ...,1
6,@azulaguard She was talking about the lore of ...,0
7,@Qiyanarbz refunded it after i flopped in 3 ga...,0
8,been playing this game for about a mont...,0
9,@wunderphilia Congratulations !!! 🥰 I have man...,1


In [ ]:
# ENTRENAR RECOMENDADOR

# Importar .csv (ejemplo de entrenamiento en clase)
import pandas as pd

trainingData = pd.read_csv('semeval-2017-train.csv', delimiter='	')
trainingData = trainingData.head(1000)
trainingData['label'].value_counts()

# Preprocesar datos de entrenamiento
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()
preprocessedText = []

for row in trainingData.itertuples():
    text = word_tokenize(row[2]) 
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    preprocessedText.append(text)

preprocessedData = trainingData
preprocessedData['processed_text'] = preprocessedText

# crear bolsa de palabras
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])

# Entrenamiento algoritmo de clasificación
from sklearn import svm
svc = svm.SVC(kernel='linear') #Modelo de clasificación

X_train = textsBoW #Documentos
Y_train = trainingData['label'] #Etiquetas de los documentos 
svc.fit(X_train, Y_train) #Entrenamiento

# Carga y preprocesado de los documentos de test
testData = pd.read_csv('semeval-2017-test.csv', delimiter='	')
testData = testData.head(100)

ps = PorterStemmer()
preprocessedText = []

for row in testData.itertuples():
    text = word_tokenize(row[2]) 
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedDataTest = testData
preprocessedDataTest['processed_text'] = preprocessedText
textsBoWTest= bagOfWordsModel.transform(preprocessedDataTest['processed_text'])

# Clasificacion de los documentos de test
X_test = textsBoWTest 
predictions = svc.predict(X_test)

# Evaluación de la predicción
from sklearn.metrics import classification_report

Y_test = testData['label']

print (classification_report(Y_test, predictions))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(X_train, Y_train) 
predictions = neigh.predict(X_test) 

print (classification_report(Y_test, predictions))


FileNotFoundError: ignored

In [ ]:
# Valorar texto sobre campeon
# (Ejecutar celda de entrenamiento antes!)

def valorar_sentimientos(texto):
  #preprocesar texto
  ps = PorterStemmer()
  pre_proc = word_tokenize(texto)
  pre_proc = " ".join(pre_proc)
  pre_proc = bagOfWordsModel.transform([pre_proc])

  #valorar sentimientos texto (-1 = negativo, 0 = neutral, 1 = positivo)
  return svc.predict(pre_proc)[0];

#recorrer tuits y valorarlos
positivos = 0
negativos = 0
for t in tuits:
  p = valorar_sentimientos(t)
  if (p == 1)
    positivos = positivos + 1
  if (p == -1)
    negativos = negativos + 1

print(negativos)
print(positivos)

# Streamlit + ngrok

In [12]:
!pip install streamlit
!pip install pyngrok
# Es necesario hacer un restart del runtime [RESTART RUNTIME]

     |████████████████████████████████| 7.5MB 6.9MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 4.5MB 34.3MB/s 
     |████████████████████████████████| 163kB 35.1MB/s 
     |████████████████████████████████| 112kB 39.7MB/s 
     |████████████████████████████████| 122kB 39.2MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=06644ff56e02f62978b69b63a3b57c411b35a4f5ad1fe255a4b1a3ca09096e77
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-5.0.1-cp36-none-any.whl size=18822 sha256=8e72d29e1d7e2d42f840278707f5baf82f01253e01dedbe4aa1aa3d017449a0f
  Stored in directory: /root/.cache/pip/wheels/94/01/05/d39efb8f6b40a411354b4168ca9dda99e6f8d586e458e97551
Successfully built pyngrok


In [4]:
%%writefile app.py
import streamlit as st
PAGE_CONFIG = {"page_title":"ABPerros","page_icon":":smiley:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)


def main():
	st.title("Awesome Streamlit for ML")
	st.subheader("How to run streamlit from colab")

  st.write("Here's our first attempt at using data to create a table:")
  st.write(pd.DataFrame({
      'first column': [1, 2, 3, 4],
      'second column': [10, 20, 30, 40]
  }))

	menu = ["Home","About"]
	choice = st.sidebar.selectbox('Menu',menu)
	if choice == 'Home':
		st.subheader("Streamlit From Colab")	



if __name__ == '__main__':
	main()

Overwriting app.py


In [5]:
!streamlit run app.py&>/dev/null& 

In [7]:
# Setup a tunnel to the streamlit port 8501
from pyngrok import ngrok
public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "http://0b0b13e58037.ngrok.io" -> "http://localhost:8501">

In [ ]:
!pgrep streamlit

In [ ]:
!ps -eaf | grep streamlit

In [ ]:
#Shutdown
!kill 373

In [6]:
ngrok.kill()

# FIN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('PandasMyNewLegend.ipynb')